In [2]:
import requests

In [3]:
requests.get('http://127.0.0.1:8001/status').json()

{'status': 'OK'}

In [6]:
requests.get('http://127.0.0.1:8001/experiment').json()

[{'experiment_id': 'c273e944-f26d-42b1-8947-2ca7753330c0'}]

In [5]:
test_data = {
    'a': {'type': 'log', 'parameters': dict(lower=0.01, upper=1.0)},
}
new_experiment = requests.post('http://127.0.0.1:8001/experiment', json=test_data).json()
new_experiment_id = new_experiment['experiment_id']
new_experiment_id

'c273e944-f26d-42b1-8947-2ca7753330c0'

In [7]:
requests.get(f'http://127.0.0.1:8001/experiment/41444{new_experiment_id}').json()

{'status': 'Experiment with such ID not Found'}

In [11]:
requests.get(f'http://127.0.0.1:8001/experiment/{new_experiment_id}/ask').json()

{'a': 0.05769609938182262}

In [67]:
import uuid

In [68]:
i = uuid.uuid4()

In [69]:
d = {i: 1}

In [70]:
d

{UUID('292207ea-cfc5-4697-9325-831ac4663164'): 1}

In [73]:
str(i)

'292207ea-cfc5-4697-9325-831ac4663164'